In [1]:
import numpy as np
import random
import threading
import warnings
import os
import sys
import nltk
sys.path.append('../../common/')

from train_log import train_log

os.environ["CUDA_VISIBLE_DEVICES"]="1,2" 
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
warnings.filterwarnings("ignore")
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.FATAL)

FILE_NAME = 'News_Category_Dataset_v2.json'

from gensim.test.utils import datapath, get_tmpfile
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

glove_file = datapath('/home/ubuntu/Notebooks/GloVe/glove.6B.100d.txt')
word2vec_glove_file = get_tmpfile("glove.6B.100d.word2vec.txt")
glove2word2vec(glove_file, word2vec_glove_file)

model = KeyedVectors.load_word2vec_format(word2vec_glove_file)

ckpt_dir_cnn = './ckpt_save_8/'
ckpt_dir_this = './ckpt_save/'
if not os.path.exists(ckpt_dir_this):
    os.makedirs(ckpt_dir_this)
log_dir = './log/'
log = train_log(log_dir)
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

CONTINUE = 0
start_step = 0


In [4]:
from NewsCategoryData import NewsCategory

def getvector(word, model=model):
    if model.vocab.get(word.lower(),"NaN") == "NaN":
        lst = nltk.stem.LancasterStemmer()
        if model.vocab.get(lst.stem(word).lower(),"NaN") == "NaN":
            if word[-1] == "s" and model.vocab.get(word[:-1].lower(),"NaN") != "NaN":
                return model[word[:-1].lower()]
            else:
                return model["unk"]
        else:
            return model[lst.stem(word).lower()]
    else:
        return model[word.lower()]

def word2vector(reviews, model=model, max_length=1000):
    vector_data = np.zeros((len(reviews), max_length,100))
    x_length = []
    i = 0
    for review in reviews:
        j = 0 
        if len(review) > max_length:
            print("The length of the reviews is %dwhich is larger than max_length (%d)"%(len(review),max_length))
            print(review)
            print("-"*10)
        for word in review:
            vector_data[i,j] = getvector(word,model)
            if str(vector_data[i,j,0])=='nan':
                print(word)
                break
            j += 1
        x_length.append(j)
        i += 1
    return vector_data, np.asarray(x_length)

def getvector2(word, model=model):
    if model.vocab.get(word.lower(),"NaN") == "NaN":
        lst = nltk.stem.LancasterStemmer()
        if model.vocab.get(lst.stem(word).lower(),"NaN") == "NaN":
            if word[-1] == "s" and model.vocab.get(word[:-1].lower(),"NaN") != "NaN":
                #print(word[:-1])
                return word[:-1].lower()
            else:
                return "unk"
        else:
            return lst.stem(word).lower()
    else:
        return word.lower()
batch_size = 64
n_classes = 41


In [30]:
batch_size = 64
n_classes = 41
graph_combined = tf.Graph()
with graph_combined.as_default():
    with tf.variable_scope('Combined_Input'):
        x_logits = tf.placeholder(tf.float32, [None, n_classes], name="x_logits")
        x_cnn = tf.nn.softmax(x_logits, name = "x_cnn")
        x_naive_bayes = tf.placeholder(tf.float32, [None,n_classes], name="x_naive_bayes")
        y_ = tf.placeholder(tf.int64, [None],  name="y_")

    with tf.variable_scope('Combined_Output'):
        w = tf.Variable(tf.random_uniform([n_classes*2, n_classes],0,1.0),name="w")
        b= tf.Variable(tf.zeros([n_classes]),name="b")
        y = tf.add(tf.matmul(tf.concat([x_cnn,x_naive_bayes],axis=1), w), b,name="y")
        #print(y.shape)

    with tf.variable_scope('Combined_Loss'):
        cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y,
                                                                                      labels=y_, 
                                                                                      name="cross_entropy_loss"))
        #cross_entropy_3 = tf.cast(tf.one_hot(y_-1,n_classes),tf.float32)*tf.log(tf.nn.softmax(y))
        
    with tf.variable_scope('Combined_Prediction'):
        prediction = tf.argmax(tf.nn.softmax(y),1, name="prediction")
        cal_accuracy = tf.reduce_mean(tf.cast(tf.equal(prediction,y_), tf.float32),name="cal_accuracy")

    with tf.variable_scope('Combined_Train'):
        #train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)
        train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

In [39]:
graph_cnn = tf.Graph()
with graph_cnn.as_default(): 
    saver = tf.train.import_meta_graph(ckpt_dir_cnn+'news_category-200000.meta')
    input_x = graph_cnn.get_tensor_by_name("Input/x:0")
    input_y = graph_cnn.get_tensor_by_name("Input/y_:0")
    x_length = graph_cnn.get_tensor_by_name("Input/x_length:0")
    output_y = graph_cnn.get_tensor_by_name("Output/y:0")

In [42]:
import Run_Prediction as rp
max_length = 100

cnn_logits_list = []
naive_bayes_input_list = []
label_list = [] 
with graph_cnn.as_default(): 
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess, coord)

        data = NewsCategory(batch_size=batch_size,max_length=max_length)
        #load the pretrained cnn model
        model_file=tf.train.latest_checkpoint(ckpt_dir_cnn)
        saver.restore(sess,model_file)
        #load naive bayes model
        word_matrix_file = "naive_bayes_word_matrix.csv"
        word_matrix, word_list = rp.read_naive_bayes_word_vector(word_matrix_file)

        try:        
            i = start_step
            avg_accuracy = 0
            avg_loss = 0
            while i < int(data.max_recorder/batch_size)+1:    
                batch_data, batch_label = data.get_batch_data()  
                batch_data_vec, data_length = word2vector(batch_data, model,max_length)
                
                cnn_logits = sess.run([output_y],
                                      feed_dict={input_x:batch_data_vec, 
                                                 input_y:batch_label, 
                                                 x_length:data_length})
                cnn_logits_list.append(cnn_logits[0])

                naive_bayes_input = rp.naive_bayes_model(batch_data,word_matrix, word_list)
                naive_bayes_input_list.append(naive_bayes_input)
                #print(naive_bayes_input.shape)
            
                label_list.append(batch_label)
                
                '''
                print('cnn_logits',cnn_logits[0])
                if cnn_logits[0].shape != (batch_size,n_classes):
                    print(cnn_logits[0])
                print('naive_bayes_input', naive_bayes_input)
                if np.isnan(naive_bayes_input).any():
                    print(batch_data)
                '''
                i += 1
        except tf.errors.OutOfRangeError:
            print('done')
        finally:
            coord.request_stop()
            coord.join(threads)
            print('quit')


Load the naive bayes word vectors. It takes a few minutes.
quit


In [21]:
import Run_Prediction as rp

max_length = 100

with graph_combined.as_default():
    saver_the_model = tf.train.Saver(max_to_keep = 1)
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess, coord)
        train_data_len = len(cnn_logits_list)
        try:        
            i = start_step
            avg_accuracy = 0
            avg_loss = 0
            while i < 50000:    
                #train the combined model 
                loss, combined_prediction, accuracy, _ = sess.run([cross_entropy,prediction,cal_accuracy, train_step],
                                                                  feed_dict={x_logits:cnn_logits_list[i%train_data_len],
                                                                             x_naive_bayes:naive_bayes_input_list[i%train_data_len],
                                                                             y_:label_list[i%train_data_len]})          
                avg_accuracy += accuracy
                avg_loss += loss
                i += 1
                #print(".",end="")
                if i%1000 == 0:
                    print("-----------%d-----------"%i)
                    print("Accuracy at %d: %.2f%%"%(i,avg_accuracy/10))
                    print("Loss at %d：%.4f"%(i, avg_loss/1000))
                    saver.save(sess,ckpt_dir_this+'combined_new_category',global_step=i)
                    #log.add_log('train_accuracy',i, avg_accuracy/1000)
                    #log.add_log('train_loss',i, avg_loss/1000)
                    #log.SaveToFile() 
                    avg_accuracy = 0
                    avg_loss = 0
        except tf.errors.OutOfRangeError:
            print('done')
        finally:
            coord.request_stop()
            coord.join(threads)
            print('quit')

-----------1000-----------
Accuracy at 1000: 3.51%
Loss at 1000：3.6749
-----------2000-----------
Accuracy at 2000: 13.40%
Loss at 2000：3.4534
-----------3000-----------
Accuracy at 3000: 22.24%
Loss at 3000：3.2502
-----------4000-----------
Accuracy at 4000: 34.13%
Loss at 4000：3.0539
-----------5000-----------
Accuracy at 5000: 43.58%
Loss at 5000：2.8775
-----------6000-----------
Accuracy at 6000: 44.74%
Loss at 6000：2.7133
-----------7000-----------
Accuracy at 7000: 45.52%
Loss at 7000：2.5576
-----------8000-----------
Accuracy at 8000: 46.37%
Loss at 8000：2.4193
-----------9000-----------
Accuracy at 9000: 47.39%
Loss at 9000：2.2921
-----------10000-----------
Accuracy at 10000: 48.71%
Loss at 10000：2.1732
-----------11000-----------
Accuracy at 11000: 50.71%
Loss at 11000：2.0698
-----------12000-----------
Accuracy at 12000: 52.34%
Loss at 12000：1.9739
-----------13000-----------
Accuracy at 13000: 55.15%
Loss at 13000：1.8744
-----------14000-----------
Accuracy at 14000: 56.77%

In [32]:
import Run_Prediction as rp
max_length = 100
#np.set_printoptions(precision=4,suppress =False)

data = NewsCategory(batch_size=batch_size,max_length=max_length,shuffle=False)
cnn_logits_list = []
naive_bayes_input_list = []
label_list = [] 
with graph_cnn.as_default(): 
    with tf.Session() as sess:
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess, coord)    

        #load the pretrained cnn model
        model_file=tf.train.latest_checkpoint(ckpt_dir_cnn)
        saver.restore(sess,model_file)

        #load naive bayes model
        word_matrix_file = "naive_bayes_word_matrix.csv"
        word_matrix, word_list = rp.read_naive_bayes_word_vector(word_matrix_file)

        try:        
            i = start_step
            avg_accuracy = 0
            while i < int(data.max_recorder/batch_size)+1:    
                batch_data, batch_label = data.get_batch_data()  
                batch_data_vec, data_length = word2vector(batch_data, model,max_length)

                cnn_logits = sess.run([output_y],feed_dict={input_x:batch_data_vec, input_y:batch_label, x_length:data_length})
                cnn_logits_list.append(cnn_logits[0])

                naive_bayes_input = rp.naive_bayes_model(batch_data,word_matrix, word_list)
                naive_bayes_input_list.append(naive_bayes_input)

                label_list.append(batch_label)
                i += 1
        except tf.errors.OutOfRangeError:
            print('done')
        finally:
            coord.request_stop()
            coord.join(threads)
            print('quit')        


Load the naive bayes word vectors. It takes a few minutes.
quit


In [36]:
avg_accuracy = 0
with graph_combined.as_default(): 
    saver = tf.train.Saver(max_to_keep = 1)
    '''
    saver = tf.train.import_meta_graph(ckpt_dir_this+'combined_new_category-50000.meta')
    
    x_logits = graph.get_tensor_by_name("Combined_Input/x_logits:0")
    x_naive_bayes = graph.get_tensor_by_name("Combined_Input/x_naive_bayes:0")
    y_ = graph.get_tensor_by_name("Combined_Input/y_:0")
    prediction = graph.get_tensor_by_name("Combined_Prediction/prediction:0")
    cal_accuracy = graph.get_tensor_by_name("Combined_Prediction/cal_accuracy:0")
    '''
    with tf.Session() as sess:
        model_file=tf.train.latest_checkpoint(ckpt_dir_this)
        print(model_file)
        saver.restore(sess, model_file)   
        print('Restore model.')
        for i in range(len(cnn_logits_list)):
            combined_prediction, accuracy= sess.run([prediction,cal_accuracy],
                                                    feed_dict={x_logits:cnn_logits_list[i],
                                                               x_naive_bayes:naive_bayes_input_list[i],
                                                               y_:label_list[i]})
            avg_accuracy += accuracy
        print("Accuracy: %.2f%%"%(avg_accuracy/i*100))

./ckpt_save/combined_new_category-50000
Restore parameters
Accuracy: 80.07%
